In [ ]:
!pip install pysrt

# Imports

In [1]:
import pandas as pd
import glob
import json
from tqdm import tqdm
import re
from datetime import datetime
import gensim
import logging
import os
import shutil
import pysrt

In [2]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore


kaggle datasets download -d goweiting/ted-talks-transcript    
Use the Kaggle API to download dataset files.      
https://github.com/Kaggle/kaggle-api    

https://www.kaggle.com/goweiting/ted-talks-transcript/data

Doc2vec   
https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

In [ ]:
!ls -lah ~/.kaggle/datasets/goweiting/ted-talks-transcript/

In [ ]:
#!unzip /Users/nidhin/.kaggle/datasets/goweiting/ted-talks-transcript/tedDirector.zip

In [3]:
base_path= os.path.expanduser("~/.kaggle/datasets/goweiting/ted-talks-transcript")

In [ ]:
#shutil.rmtree(base_path, ignore_errors=True)
#os.makedirs(base_path)

In [ ]:
!ls ~/.kaggle/datasets/goweiting/ted-talks-transcript


In [ ]:
!kaggle datasets download -d goweiting/ted-talks-transcript -p {base_path}/

In [ ]:
!ls ~/.kaggle/datasets/goweiting/ted-talks-transcript

In [4]:
!ls {base_path}

subtitles                tedDirector.stats.csv    ted_metadata_kaggle.csv
ted-talks-transcript.zip tedDirector.zip          ted_metadata_youtube.csv


In [ ]:
!unzip -q -o {base_path}/ted-talks-transcript.zip -d {base_path}
!unzip -q -o {base_path}/tedDirector.zip -d {base_path}

In [ ]:
print(f"{base_path}subtitles/*.info.json")

In [5]:
meta_data_files = glob.glob(f"{base_path}/subtitles/*/*.info.json")

In [6]:
len(meta_data_files)

2531

In [7]:
def parse_video_info(metadata):
    keys = ["webpage_url", "view_count", "upload_date", "title", "thumbnail" 
            , "duration" , "dislike_count" , "like_count"
            , "description", "categories" ,"id", "average_rating"
            ]
    
    new_dict = {}
    for key in keys:
        new_dict[key] = metadata.get(key,None)
        
    return new_dict

def get_metadata(meta_data_files) -> pd.DataFrame:
    parsed_metadatas = []
    for file in tqdm(meta_data_files):
        with open(file) as f:
            content = json.load(f)
        
        parsed_metadata = parse_video_info(content)
        parsed_metadatas.append(parsed_metadata)

    return pd.DataFrame(parsed_metadatas)


In [8]:
?pd.to_datetime

In [9]:
metadata_df = get_metadata(meta_data_files)
metadata_df['upload_date']=pd.to_datetime(metadata_df['upload_date'],format='%Y%m%d')

100%|██████████| 2531/2531 [00:00<00:00, 3736.85it/s]


In [11]:
metadata_df.head(20)

In [12]:
subtitle_files = glob.glob(f"{base_path}/subtitles/*/*.en.vtt")

In [13]:
subtitle_files[0:2]

['/Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/Jc2F3-nawnI/Jc2F3-nawnI.en.vtt',
 '/Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/PdxPCeWw75k/PdxPCeWw75k.en.vtt']

In [14]:
def parse_subtitle(file_name) -> str:
    subs = pysrt.open(file_name)
    content = ""
    for s in subs:
        content = content + s.text_without_tags + "\n"
    return content

def get_subtitles(subtitle_files) -> pd.DataFrame :
    parsed_subtitles = []
    for file in tqdm(subtitle_files):
        
        res = re.match('.*/(.*).en.vtt',file)
        talk_id  = res.group(1)
        content = parse_subtitle(file)
        
        parsed_subtitles.append ({'id': talk_id , 'transcript': content})
        
    return pd.DataFrame(parsed_subtitles)
        

In [15]:
subtitle_files_df = get_subtitles(subtitle_files)

100%|██████████| 2490/2490 [00:21<00:00, 117.79it/s]


In [16]:
subtitle_files_df.head()

In [17]:
len(metadata_df)

2531

In [18]:
len(subtitle_files_df)

2490

In [19]:
df = pd.merge(metadata_df,subtitle_files_df, on='id')

In [20]:
df.head()

In [21]:
df.to_csv("talks.csv.gzip",compression='gzip',index=False)